In [23]:
import warnings

import numpy as np
import scipy.integrate

import bokeh.application
import bokeh.application.handlers
import bokeh.layouts
import bokeh.models
import bokeh.palettes
import bokeh.plotting

import colorcet

from jsfunctions import jsfuns

In [29]:
gaus_funs = {

"gaussian_pulse": """
function sPulse(t, t0, tau) {
    return Math.exp(-4.0 * Math.pow((t - t0) / tau, 2));
}


function callback() {
    let t0 = t0_slider.value;
    let tau = tau_slider.value;
    let t = cds.data['t'];
    let s = cds.data['s'];

    for (let i = 0; i < s.length; i++) {
        s[i] = sPulse(t[i], t0, tau);
    }

    cds.change.emit();
}
"""   
}

In [33]:
def gaussian_pulse():
    """Make a plot of a Gaussian pulse/
    """
    # t/s data for plotting
    t_0 = 4.0
    tau = 2.0
    t = np.linspace(0, 10, 200)
    s = np.exp(-4 * (t - t_0) ** 2 / tau ** 2)

    # Place the data in a ColumnDataSource
    cds = bokeh.models.ColumnDataSource(dict(t=t, s=s))

    # Build the plot
    p = bokeh.plotting.figure(
        frame_height=200,
        frame_width=400,
        x_axis_label="time",
        y_axis_label="input signal",
        x_range=[0, 10],
        y_range=[-0.02, 1.1],
    )
    p.line(source=cds, x="t", y="s", line_width=2)

    t0_slider = bokeh.models.Slider(
        title="t₀", start=0, end=10, step=0.01, value=4.0, width=150
    )
    tau_slider = bokeh.models.Slider(
        title="τ", start=0, end=10, step=0.01, value=2.0, width=150
    )

    # JavaScript callback
    js_code = gaus_funs["gaussian_pulse"] + "callback()"
    callback = bokeh.models.CustomJS(
        args=dict(cds=cds, t0_slider=t0_slider, tau_slider=tau_slider), code=js_code,
    )
    t0_slider.js_on_change("value", callback)
    tau_slider.js_on_change("value", callback)

    # Lay out and return
    return bokeh.layouts.row(
        p, bokeh.models.Spacer(width=30), bokeh.layouts.column(t0_slider, tau_slider)
    )

In [34]:
js_code = jsfuns["gaussian_pulse"] + "callback()"


In [35]:
bokeh.io.show(gaussian_pulse())